In [2]:
import keras

import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.models import load_model

import csv
from csv import writer
from csv import DictWriter

import glob

import nltk, re, string, collections
from nltk import word_tokenize 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
sentiment_corpus = []

In [4]:
product_comments = pd.read_csv("../input/productcomments/data.csv", encoding = "utf-8", error_bad_lines=False)

for y,d in zip(product_comments['Yorum'],product_comments['Duygu']):
    y = [i.strip().lower() for i in y.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
    if len(y) > 1:
        sentiment_corpus.append([y, d])

In [5]:
path = "../input/sa-imdb/SA_IMDB_translated_TR/TEST/negTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
        if len(l) > 1:
            sentiment_corpus.append([l, 0])
print("ok")    
path = "../input/sa-imdb/SA_IMDB_translated_TR/TEST/posTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
        if len(l) > 1:
            sentiment_corpus.append([l, 1])
print("ok")        

path = "../input/sa-imdb/SA_IMDB_translated_TR/TRAIN/negTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
        if len(l) > 1:
            sentiment_corpus.append([l, 0])
print("ok")        

path = "../input/sa-imdb/SA_IMDB_translated_TR/TRAIN/posTr/*.txt"
all_files = glob.glob(path)
for file in all_files:
    lines = open(file).readlines()
    for l in lines:
        l = [i.lower() for i in l.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
        if len(l) > 1:
            sentiment_corpus.append([l, 1])

ok
ok
ok


In [6]:
reviews = open("../input/sa-tr-reviews/reviews.neg").readlines()
for x in reviews:
    x = [i.lower() for i in x.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
    if len(x) > 1:
        sentiment_corpus.append([x, 0])

reviews = open("../input/sa-tr-reviews/reviews.pos").readlines()
for x in reviews:
    x = [i.lower() for i in x.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", i) and len(i) > 1 and i not in stopwords.words('turkish')]
    if len(x) > 1:
        sentiment_corpus.append([x, 1])

In [7]:
my_array =np.array(sentiment_corpus)
df = pd.DataFrame(my_array, columns = ['text','sentiment'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [8]:
df

,text,sentiment
0,"[iyi, bir, ürün]",1
1,"[gerçekten, bilene, iyi, orta, segmet, bilecek...",1
2,"[kargo, yavaş, gelmedi, gün, sonra, yola, heps...",1
3,"[daha, önce, kötü, tecrübem, samsung, türkiye,...",1
4,"[fiyatinin, dusmesini, bekliyodum, gecen, haft...",1
...,...,...
277473,"[ürün, hediye, paketi, içerisinde, kol, saati,...",1
277474,"[ürün, göndermiş, olduğunuz, hediye, ince, bir...",1
277475,"[burada, tarihinde, ilk, yorum, bunun, göz, ön...",1
277476,"[güzel, bir, ürün, mağazaya, hepsiburadaya, te...",1


In [9]:
# df = pd.read_csv("../input/sentiment-analysis-big-corpus/sentiment-analysis-bigcorpus.csv", encoding = "utf-8", error_bad_lines=False)

In [10]:
# df

In [11]:
# df = pd.DataFrame(df, columns = ['text','sentiment'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277478 entries, 0 to 277477
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       277478 non-null  object
 1   sentiment  277478 non-null  object
dtypes: object(2)
memory usage: 4.2+ MB


In [13]:
print(df[df['sentiment'] == 1].size)
print(df[df['sentiment'] == 0].size)
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

476332
78624


In [14]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 308, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 308, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 123)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(185910, 308) (185910, 2)
(91568, 308) (91568, 2)


In [16]:
# load model
model = load_model('../input/lstm-sa-model-last/lstm_sa_model (5).h5')
# summarize model.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 308, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 308, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [17]:
# model.fit(X_train, Y_train, epochs = 7, batch_size=200, verbose = 1)

In [18]:
# model.save('lstm_sa_model.h5')

In [19]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 256)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

352/352 [==============================] - 26s 72ms/step - loss: 0.2172 - accuracy: 0.9187
score: 0.22
acc: 0.92


In [20]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=256,verbose = 0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 97.52895752895753 %
neg_acc 59.512195121951216 %


In [21]:
print("../input/tbmmcorpusdonem2027/NLP/donem20/yıl1/44.txt\n")

demo = ["Söz konusu davadan vazgeçilmesi halinde İstanbul Bankasına ait tüm borçların hazine tarafından karşılanması kesinlikle söz konusu değildir."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem20/yıl1/44.txt

['Söz konusu davadan vazgeçilmesi halinde İstanbul Bankasına ait tüm borçların hazine tarafından karşılanması kesinlikle söz konusu değildir.']
[[   0  448  686 1419 1041  263   29  448  686 1077]]
1/1 - 0s
positive


In [22]:
print("../input/tbmmcorpusdonem2027/NLP/donem21/yıl3/124.txt\n")

demo = ["Amaç, millete hizmet değil mi?! Biz, bu kararla, en azından, önemli bir şeyin herkes tarafından anlaşılmış olmasından dolayı memnunuz."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem21/yıl3/124.txt

['Amaç, millete hizmet değil mi?! Biz, bu kararla, en azından, önemli bir şeyin herkes tarafından anlaşılmış olmasından dolayı memnunuz.']
[[   0  508   68  412    3  176  161    1  463  202 1741]]
1/1 - 0s
positive


In [23]:
print("../input/tbmmcorpusdonem2027/NLP/donem22/yıl5/119.txt\n")

demo = [" Efendim, İç Tüzük'ün 69'uncu maddesine göre, Sayın Hatip Anavatan Grubunu suçlayarak söze başladığı için, söz almak istiyorum."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem22/yıl5/119.txt

[" Efendim, İç Tüzük'ün 69'uncu maddesine göre, Sayın Hatip Anavatan Grubunu suçlayarak söze başladığı için, söz almak istiyorum."]
[[   0   31  448  118 1685]]
1/1 - 0s
positive


In [24]:
print("../input/tbmmcorpusdonem2027/NLP/donem23/yıl5/24.txt\n")

demo = ["Türkiye Büyük Millet Meclisinin 90’ıncı yılında, Türk kadınının belediye meclislerine seçme ve seçilme hakkını kazanışının 80’inci yılında ve milletvekili seçme ve seçilme hakkını elde edişinin 76’ncı yılında durum kadınlar ve demokrasimiz açısından vahimdir."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem23/yıl5/24.txt

['Türkiye Büyük Millet Meclisinin 90’ıncı yılında, Türk kadınının belediye meclislerine seçme ve seçilme hakkını kazanışının 80’inci yılında ve milletvekili seçme ve seçilme hakkını elde edişinin 76’ncı yılında durum kadınlar ve demokrasimiz açısından vahimdir.']
[[   0 1269   17 1300   66   66   66  372  688   66]]
1/1 - 0s
positive


In [25]:
print("../input/tbmmcorpusdonem2027/NLP/donem24/yıl2/118.txt\n")

demo = ["Milletvekili Adnan Keskin’in, Denizli’de dolu nedeniyle üzüm bağlarının uğradığı zarara ve üreticilerin mağduriyetine ilişkin Gıda, Tarım ve Hayvancılık Bakanından yazılı soru önergesi"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem24/yıl2/118.txt

['Milletvekili Adnan Keskin’in, Denizli’de dolu nedeniyle üzüm bağlarının uğradığı zarara ve üreticilerin mağduriyetine ilişkin Gıda, Tarım ve Hayvancılık Bakanından yazılı soru önergesi']
[[   0  301  491   66   66 1078]]
1/1 - 0s
positive


In [26]:
print("../input/tbmmcorpusdonem2027/NLP/donem26/yıl2/15.txt\n")

demo = ["Bursa Milletvekili Orhan Sarıbal’ın, hiçbir zaman bu kadar faşizan, bu kadar baskıcı bir düzen görülmediğine ve basına, insan hayatına bu kadar müdahalenin olmadığına ilişkin açıklaması"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem26/yıl2/15.txt

['Bursa Milletvekili Orhan Sarıbal’ın, hiçbir zaman bu kadar faşizan, bu kadar baskıcı bir düzen görülmediğine ve basına, insan hayatına bu kadar müdahalenin olmadığına ilişkin açıklaması']
[[  0  57  20   3   7   3   7   1  66 226   3   7]]
1/1 - 0s
negative


In [27]:
print("../input/tbmmcorpusdonem2027/NLP/donem27/yıl4/17.txt\n")

demo = ["Daha niceleri iktidarın baskı ve korkusuna direnemeyen hâkim ve savcılar yüzünden hâlen cezaevindedir."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

../input/tbmmcorpusdonem2027/NLP/donem27/yıl4/17.txt

['Daha niceleri iktidarın baskı ve korkusuna direnemeyen hâkim ve savcılar yüzünden hâlen cezaevindedir.']
[[  0 187  66  66 889]]
1/1 - 0s
positive


In [28]:
demo = ["Bu bir demo cümlesidir. Bakalım sonuç ne olacak?"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Bu bir demo cümlesidir. Bakalım sonuç ne olacak?']
[[  0   3   1 243 182 588]]
1/1 - 0s
positive


In [40]:
demo = ["Bu ürünü hiç sevmedim param boşa gitti tavsiye etmiyorum."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Bu ürünü hiç sevmedim param boşa gitti tavsiye etmiyorum.']
[[  0   3  25 892 493 886   8]]
1/1 - 0s
negative


In [30]:
demo = ["Gerçekten çok sevindim, kutlarım!"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Gerçekten çok sevindim, kutlarım!']
[[  0  12 178]]
1/1 - 0s
positive


In [31]:
demo = ["Ben bu üründen hiç bir şey anlamadım iyi mi kötü mü bilemedim."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Ben bu üründen hiç bir şey anlamadım iyi mi kötü mü bilemedim.']
[[  0  22   3 380 892   1   2 412  21]]
1/1 - 0s
negative


In [32]:
demo = ["Bu günlerde çok kararsızım."]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Bu günlerde çok kararsızım.']
[[  0   3 932 178]]
1/1 - 0s
positive


In [33]:
demo = ["Bu cümleden beklentim pozitif yönde bir sonuç vermesi. Kafa karışıklığı için üzgün olduğumu belirtmek istiyorum. Mesela bu ürün gerçekten berbat desem sonucu etkiler miyim? Bakalım!"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Bu cümleden beklentim pozitif yönde bir sonuç vermesi. Kafa karışıklığı için üzgün olduğumu belirtmek istiyorum. Mesela bu ürün gerçekten berbat desem sonucu etkiler miyim? Bakalım!']
[[   0    3    1  243 1842  663 1232 1685    3    4   12  570  873]]
1/1 - 0s
negative


In [34]:
demo = ["Bu cümleden beklentim negatif yönde bir sonuç vermesi. Kafa karışıklığı için mutlu olduğumu belirtmek istiyorum. Mesela bu ürün gerçekten harika desem sonucu etkiler miyim? Bakalım!"]
print(demo)
demo = tokenizer.texts_to_sequences(demo)
demo = pad_sequences(demo, maxlen=len(demo[0])+1, dtype='int32', value=0)
print(demo)
sentiment = model.predict(demo,batch_size=256,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

['Bu cümleden beklentim negatif yönde bir sonuç vermesi. Kafa karışıklığı için mutlu olduğumu belirtmek istiyorum. Mesela bu ürün gerçekten harika desem sonucu etkiler miyim? Bakalım!']
[[   0    3    1  243 1842  663  393 1232 1685    3    4   12   34  873]]
1/1 - 0s
positive
